+++++++++++++++++++++++++++
This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]
+++++++++++++++++++++++++++

### This is a utility script used to authenticate the user with CalNet and store the resulting auth and refresh tokens to be used in other scripts.

In [7]:
import os, sys
from boxsdk.exception import BoxAPIException

function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [8]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
        f.write(refresh_token.strip())

The client id and client secret are defined in the Box application created for this notebook. Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens which are then continually refreshed.

In [9]:
# Define client ID, client secret, and developer token.
CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None

# Read app info from text file
with open('app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()

# for debugging
# print ("CLIENT_ID: ", CLIENT_ID )
# print ("CLIENT_SECRET: ", CLIENT_SECRET )
# print ("REDIRECT_URI: ", REDIRECT_URI )

CLIENT_ID:  ...

CLIENT_SECRET:  ...

REDIRECT_URI:  https://auth.berkeley.edu/cas/login



Perform autentication then create globus client. 
Place the url that is returned into a browser to authenticate with CalNet. CalNet will return a code in the resulting url. 

In [10]:
from boxsdk import OAuth2

oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    store_tokens=store_tokens
)

auth_url, csrf_token = oauth.get_authorization_url(REDIRECT_URI.strip())

print ("auth_url", auth_url)

auth_url https://app.box.com/api/oauth2/authorize?state=box_csrf_token_cz3zMI1Z1fWSnaZt&response_type=code&client_id=xxx&redirect_uri=https%3A%2F%2Fauth.berkeley.edu%2Fcas%2Flogin


Place the code provided in the line below before executing this cell.

In [11]:
#access_token, refresh_token = oauth.authenticate('paste code here...')
access_token, refresh_token = oauth.authenticate('')
store_tokens(access_token, refresh_token)

Create a client to Box and verify that the authorization is valid.
Verify client by retrieving the name of the users root folder in Box.

In [12]:
from boxsdk import Client

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

folder name:  All Files
